<a href="https://colab.research.google.com/github/ucfilho/marquesgabi/blob/master/Calculo_017_out_31_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# carrega as bibliotecas do python

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import os 
import cv2
import zipfile
from skimage import data
from skimage import filters
from skimage import exposure
import skimage.feature as sk
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.metrics.cluster import entropy
from sklearn.metrics.cluster import homogeneity_score


In [3]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip


--2019-10-31 03:15:26--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip.3’

Sugar_set_30_2019.z 100%[===================>]  18.68M  5.42MB/s    in 3.4s    

2019-10-31 03:15:30 (5.42 MB/s) - ‘Sugar_set_30_2019.zip.3’ saved [19584173/19584173]



In [0]:

file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall() 

In [0]:
# all images are resized in variable x, are in gray in variable p and in variable w are dataframe
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=5 # trocar por 64 qdo acabar a analise

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol))
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
Nfotos=len(img_name)
GLCM=np.zeros((Nfotos, 14))   
Nrow=Size
Ncol=Size
#print(GLCM.shape)
#Soma=np.sum(pp[k,:])

G=Ncol-1


  





for k in range(Nfotos):
  Pdif=[] 
  Psom=[]
  p=pp[k,:].reshape(Size,Size)
  Soma=0
  for i in range(Size):
    for j in range(Size):
      Soma=Soma+p[i,j]
   
  for i in range(Size):
    for j in range(Size):
      p[i,j]=p[i,j]/Soma

  for kr in range(2*G+1):
    Soma_soma=0
    for i in range(Nrow):
      for j in range(Ncol):
        if((i+j)==kr):
          Soma_soma=Soma_soma+p[i,j]
    Psom.append(Soma_soma)

  for kr in range(G+1):
    Soma_dif=0
    for i in range(Nrow):
      for j in range(Ncol):
        if((abs(i-j))==kr):
          Soma_dif=Soma_dif+p[i,j]
    Pdif.append(Soma_dif)


  for i in range(Nrow):
    for j in range(Ncol):
      #print("i=%d j=%d k=%d"%(i,j,k))
      GLCM[k,0]=GLCM[k,0]+p[i,j]**2 #segundo momento angular
      GLCM[k,1]=GLCM[k,1]+(i-j)**2*p[i,j] #contraste


      MIx=0;MIy=0
      for ii in range(Nrow):
        for jj in range(Ncol):
          MIy=MIy+jj*p[ii,jj]
          MIx=MIx+ii*p[ii,jj]
      Sx2=0;Sy2=0;
      for ii in range(Nrow):
        for jj in range(Ncol):
          Sx2=Sx2+(ii-MIx)**2*p[ii,jj]
          Sy2=Sy2+(jj-MIy)**2*p[ii,jj]
      Sx=Sx2**0.5
      Sy=Sy2**0.5


      GLCM[k,2]=GLCM[k,2]+((i*j)*p[i,j]-MIx*MIy)/(Sx*Sy) #Correlacao
      # CONFERIR CORRELACAO EM OUTROS

      GLCM[k,3]=GLCM[k,3]+(i-np.mean(p))**2*p[i,j] #Variancia

      GLCM[k,4]=GLCM[k,4]+(1/(1+(i-j)**2))*p[i,j]#INVERSA DIF MOMENTO
      
      f8=0
      
      for kr in range(len(Psom)):
        GLCM[k,5]=GLCM[k,5]+kr*Psom[kr] # conferir !!!!! ok???
        f8=f8-Psom[kr]*np.log(Psom[kr])

      for kr in range(len(Psom)):
        GLCM[k,6]=GLCM[k,6]+(kr-f8)**2*Psom[kr] # sum of variance

      GLCM[k,7]=f8  # sum entropy
      GLCM[k,8]=GLCM[k,8]-p[i,j]*np.log(p[i,j]) # entropia

      for kr in range(len(Pdif)):
        GLCM[k,9]=GLCM[k,9]+(kr)**2*Pdif[kr] # variance difference
        GLCM[k,10]=GLCM[k,10]-Pdif[kr]*np.log(Pdif[kr]) # difference entropy


In [0]:
# ver os conceitos GLCM aqui tambem
# http://haralick.org/journals/TexturalFeatures.pdf

In [8]:
pd.set_option('display.max_columns', 30)
print(pd.DataFrame(GLCM))

           0         1          2         3         4           5           6  \
0   0.040159  4.006764 -48.388893  5.873156  0.423837  100.489498  194.240670   
1   0.041540  4.282528 -47.724904  5.762803  0.411868  100.591416  192.850445   
2   0.040287  4.030399 -49.193342  5.979967  0.424910  101.042771  195.669329   
3   0.040175  4.015663 -48.607804  6.027608  0.435603  101.157327  197.985601   
4   0.040313  4.043838 -47.746904  5.656749  0.422210   99.676608  190.426187   
5   0.040234  4.056964 -49.752577  5.948915  0.431240  102.170893  201.154237   
6   0.040177  3.895373 -49.375098  5.944445  0.435292  101.182666  197.869755   
7   0.040276  3.931919 -51.702480  6.036687  0.428104  103.019158  203.469908   
8   0.040229  3.993338 -49.941784  6.066993  0.427480  102.101840  200.906815   
9   0.040272  3.979461 -49.007498  5.888981  0.430643  101.235558  197.961370   
10  0.040134  3.934069 -48.671268  5.729726  0.432728  100.181388  192.771285   
11  0.041463  3.928237 -48.5

In [9]:
GLCM=pd.DataFrame(GLCM)
pd.set_option('display.max_columns', 30)
print(GLCM.head(5))

          0         1          2         3         4           5           6  \
0  0.040159  4.006764 -48.388893  5.873156  0.423837  100.489498  194.240670   
1  0.041540  4.282528 -47.724904  5.762803  0.411868  100.591416  192.850445   
2  0.040287  4.030399 -49.193342  5.979967  0.424910  101.042771  195.669329   
3  0.040175  4.015663 -48.607804  6.027608  0.435603  101.157327  197.985601   
4  0.040313  4.043838 -47.746904  5.656749  0.422210   99.676608  190.426187   

          7         8           9         10   11   12   13  
0  2.080038  3.216867  100.169099  38.044950  0.0  0.0  0.0  
1  2.064334  3.203164  107.063197  38.443785  0.0  0.0  0.0  
2  2.071191  3.215219  100.759986  38.140968  0.0  0.0  0.0  
3  2.083742  3.216685  100.391577  38.273419  0.0  0.0  0.0  
4  2.073301  3.214975  101.095940  38.073945  0.0  0.0  0.0  


In [10]:
import numpy as np
#from skimage.feature import haar_like_feature_coord
#from skimage.feature import draw_haar_like_feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
#import skimage.feature.greycoprops as greycoprops
image = np.array([[0, 0, 1, 1],[0, 0, 1, 1],[0, 2, 2, 2],[2, 2, 3, 3]], dtype=np.uint8)
g = greycomatrix(image, [1, 2], [0, np.pi/2], levels=6,normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[0.58333333 1.        ]
 [1.25       2.75      ]]
[[ 0.71953255  0.48571429]
 [ 0.41176471 -0.29411765]]
